<a href="https://colab.research.google.com/github/pitwegner/UTS_ML2019_Project/blob/master/A2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

* All segments have different lengths -> sliding window approach
  * maybe length of 100 or 200? I read that the window size matters a lot

## Open Questions

* Network architecture? -> LSTM, CNN, Hierarchical Attention?
  * Maybe implement multiple and compare?

In [0]:
import pandas as pd
import glob
import numpy as np
import math

In [0]:
from google.colab import drive
drive.mount('/content/drive')

activities = pd.read_csv("/content/drive/My Drive/train/activities_train.csv") # Activity Labels for Segments and Nurse ID
mocap = pd.DataFrame()
print("Reading Mocap Data")
i = 0
bar_length = 50
files = glob.glob("/content/drive/My Drive/train/mocap/segment*.csv")
for mf in files:
    i += 1
    progress = math.ceil(bar_length * i / len(files))
    print("\r", "[" + "=" * progress + " " * (bar_length - progress) + "] " + "{0:.2f}".format(100 * i / len(files)) + '%', end="")
    mocap = mocap.append(pd.read_csv(mf).ffill().bfill().fillna(0))
mocap = mocap.reset_index().drop(columns=['index','time_elapsed'])

In [0]:
mocap_normalized = (mocap-mocap.min())/(mocap.max()-mocap.min())
mocap_normalized.segment_id = mocap.segment_id
mocap = mocap_normalized
activity_arr = activities.activity_id.unique()
activity_arr.sort()

In [0]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim

torch.manual_seed(0)
np.random.seed(0)

class Dataset(data.Dataset):
  
  def __init__(self, train, labels):
        self.labels = labels
        self.data = train

  def __len__(self):
        return len(self.data)

  def __getitem__(self, index):
        X = self.data[index].drop(columns=['segment_id']).values
        sid = self.data[index].segment_id.unique()[0]
        labels = self.labels[self.labels.segment_id == sid]
        aid = labels.activity_id.values[0]
        y = activity_arr.tolist().index(aid)

        return X, y

dataset = Dataset(mocap, activities)
window_length = 200

In [0]:
class SimpleCNN(torch.nn.Module):
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.output_channels = 24
        self.hidden_parameters = 64
        
        self.output_x = int((window_length - self.kernel_size + 2 * self.padding) / self.stride) + 1
        self.output_y = int((dataset[0:1][0].shape[1] - self.kernel_size + 2 * self.padding) / self.stride) + 1
        
        self.conv1 = torch.nn.Conv2d(1, self.output_channels, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(self.output_channels * int(self.output_x / 2) * int(self.output_y / 2), self.hidden_parameters)
        self.fc2 = torch.nn.Linear(self.hidden_parameters, len(activities.activity_id.unique()))
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, self.output_channels * int(self.output_x / 2) * int(self.output_y / 2))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return(x)

In [0]:
from torch.utils.data.sampler import Sampler    

class RandomWindowSampler(Sampler):
  
  def __init__(self, indices):
    self.indices = indices
  
  def __iter__(self):
    return (slice(self.indices[i], self.indices[i] + window_length) for i in torch.randperm(len(self.indices)))
  
  def __len__(self):
    return len(self.indices)
  

In [0]:
indices = []
for sid in dataset.data.segment_id.unique():
    indices += list(dataset.data[dataset.data.segment_id == sid].index[:-window_length])

split = int(np.floor(0.15 * len(indices)))
np.random.shuffle(indices)
train_indices, val_indices, test_indices = indices[split+split:], indices[split:split+split], indices[:split]

train_sampler = RandomWindowSampler(train_indices)
val_sampler = RandomWindowSampler(val_indices)
test_sampler = RandomWindowSampler(test_indices)

def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=2)
    return(train_loader)
  
val_loader = torch.utils.data.DataLoader(dataset, batch_size=128, sampler=val_sampler, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=4, sampler=test_sampler, num_workers=2)

In [0]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [0]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
  
    print("===== HYPERPARAMETERS =====")
    print("batch_size =", batch_size)
    print("epochs =", n_epochs)
    print("learning_rate =", learning_rate)
    print("=" * 27)
    
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    training_start_time = time.time()
    
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            if inputs.shape != (32,200,87):
                # TODO: Handle leftover batches (<32)
                print(inputs, inputs.shape)
                continue
            inputs, labels = Variable(inputs.reshape((32,1,200,87))), Variable(labels)
            
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data.item()
            total_train_loss += loss_size.data.item()
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every) == 0:
                train_losses.append(running_loss / print_every)
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / len(train_loader)), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
            
                #At the end of every 10th batch, do a short pass on the validation set
                total_val_loss = 0
                for i, data in enumerate(val_loader, 0):

                    #Wrap tensors in Variables
                    inputs, labels = data
                    if inputs.shape != (128,200,87):
                        # TODO: Handle leftover batches (<128)
                        print(inputs, inputs.shape)
                        continue
                    inputs, labels = Variable(inputs.reshape((128,1,200,87))), Variable(labels)
                    #Forward pass
                    val_outputs = net(inputs)
                    val_loss_size = loss(val_outputs, labels)
                    total_val_loss += val_loss_size.data.item()

                    if i >= print_every:
                      break
                
                val_losses.append(total_val_loss / print_every)
                print("Validation loss = {:.2f}".format(total_val_loss / print_every))
                start_time = time.time()
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [0]:
CNN = SimpleCNN()
train_losses = []
val_losses = []
trainNet(CNN.double(), batch_size=32, n_epochs=1, learning_rate=0.001)

In [0]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses,
                    mode='lines',
                    name='train_loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses,
                    mode='lines',
                    name='val_loss'))

fig.show()

In [0]:
confusion_matrix = np.zeros((6,6))
for i, data in enumerate(test_loader, 0):

    #Wrap tensors in Variables
    inputs, labels = data
    if inputs.shape != (4,200,87):
        # TODO: Handle leftover batches (<4)
        print(inputs, inputs.shape)
        continue
    inputs, labels = Variable(inputs.reshape((4,1,200,87))), Variable(labels)
    #Forward pass
    val_outputs = CNN.double()(inputs)
    value, index = val_outputs[0].max(0)
    confusion_matrix[index.item(), labels[0].item()] += 1
print(confusion_matrix)

In [0]:
np.set_printoptions(suppress=True)
print(confusion_matrix)